In [ ]:
%load_ext lab_black

import h5py
import os
import numpy as np
from typing import Dict, List, Optional, Tuple

from dataclasses import dataclass
from tqdm.auto import tqdm
from scipy.signal import savgol_filter
from scipy.interpolate import interp2d
from functools import lru_cache
import lmfit as lm


import plotly.graph_objects as go
import plotly.colors as pc
import matplotlib.pyplot as plt

import sys

sys.path.append(r"C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python")
from arpes_functions import (
    fitting_functions,
    tr_functions,
)


colors = pc.qualitative.D3
angstrom = "\u212B"
degree = "\u00B0"

In [ ]:
Ag_5ml = np.array(
    [(5, 2.8, 0.045), (5, 2.2, 0.8), (5, 1.9, 100)]
)  # S1 should be "several hundred ps"
Ag_15ml = np.array(
    [(15, 2.8, 0.038), (15, 2.2, 1.5), (15, 1.9, 100)]
)  # S1 should be "several hundred ps"
Cu_multilayer = np.array(
    [
        (6, 2.8, None),
        (6, 2.0, None),
        (6, 1.7, None),
    ]
)  # No lifetimes reported for Cu(111), thickness not stated, saying 6 arbitrarily
Au_10ml = np.array([(10, 2.68, 0.15), (10, 2.11, 5), (10, 1.7, 20)])  # > 5 & > 20

In [ ]:
exciton = ["diamond", "square", "circle"]

In [ ]:
## Plot data
fig = tr_functions.thesis_fig(
    title=f"Exciton Energies",
    xaxis_title=f"Film thickness (ML)",
    yaxis_title=f"Exciton energy (eV)",
    gridlines=False,
    equiv_axes=False,
    height=700,
    width=800,
    dtick_y=0.5,
)

fig.add_trace(
    go.Scatter(
        x=Ag_5ml[:, 0],
        y=Ag_5ml[:, 1],
        name="Ag(111)",
        mode="markers",
        marker=dict(symbol=exciton, size=14, color="grey"),
    )
)

fig.add_trace(
    go.Scatter(
        x=Ag_15ml[:, 0],
        y=Ag_15ml[:, 1],
        # name="Ag(111) 15 ML",
        showlegend=False,
        mode="markers",
        marker=dict(symbol=exciton, size=14, color="grey"),
    )
)

fig.add_trace(
    go.Scatter(
        x=Cu_multilayer[:, 0],
        y=Cu_multilayer[:, 1],
        name="Cu(111) - film thickness not stated",
        mode="markers",
        marker=dict(symbol=exciton, size=14, color="orange"),
    )
)

fig.add_trace(
    go.Scatter(
        x=Au_10ml[:, 0],
        y=Au_10ml[:, 1],
        name="Au(111)",
        mode="markers",
        marker=dict(symbol=exciton, size=14, color="gold"),
    )
)


fig.update_layout(yaxis_range=(0, 3), xaxis_range=(0, 20))
fig.update_xaxes(minor_ticks="inside")
fig.update_yaxes(minor_ticks="inside")
fig.update_layout(
    legend=dict(
        x=0.65,  # x position (1 is the far right)
        y=0.22,  # y position (0 is the far bottom)
        xanchor="right",  # Horizontal alignment: 'left', 'right', 'center', 'auto'
        yanchor="top",  # Vertical alignment: 'top', 'bottom', 'middle', 'auto'
        # bordercolor="Black",  # Border color of the legend
        # borderwidth=2,  # Border width of the legend
        # bgcolor="rgba(230, 230, 150, 0.8)",  # Background color of the legend with transparency
        bgcolor=None,
        font=dict(size=18),
    ),
)
fig.show()

fig.write_image(
    r"C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Figs\CH6_tr_arpes\exciton_energies_comparison.png"
)

In [ ]:
## Plot data
fig = tr_functions.thesis_fig(
    title=f"Exciton Decay Constants",
    xaxis_title=f"Film thickness (ML)",
    yaxis_title=f"Exciton decay constant (ps)",
    gridlines=False,
    equiv_axes=False,
    height=700,
    width=800,
    # dtick_y=0.5,
)

# xval = 5
# uncertainty = 0.01
# yval = (Ag_5ml[0, 2] - uncertainty, Ag_5ml[0, 2] + uncertainty)
# fig.add_shape(
#     go.layout.Shape(
#         type="line",
#         x0=xval,
#         x1=xval,
#         y0=yval[0],
#         y1=yval[1],
#         line=dict(color="grey", width=14),
#         opacity=0.25,
#     )
# )

# xval = 5
# yval = (100, 150)
# fig.add_shape(
#     go.layout.Shape(
#         type="line",
#         x0=xval,
#         x1=xval,
#         y0=yval[0],
#         y1=yval[1],
#         line=dict(color="grey", width=14),
#         opacity=0.25,
#     )
# )

fig.add_trace(
    go.Scatter(
        x=Ag_5ml[:, 0],
        y=Ag_5ml[:, 2],
        name="Ag(111)",
        mode="markers",
        marker=dict(symbol=exciton, size=14, color="grey"),
    )
)

# xval = 15
# yval = (100, 150)
# fig.add_shape(
#     go.layout.Shape(
#         type="line",
#         x0=xval,
#         x1=xval,
#         y0=yval[0],
#         y1=yval[1],
#         line=dict(color="grey", width=14),
#         opacity=0.25,
#     )
# )

fig.add_trace(
    go.Scatter(
        x=Ag_15ml[:, 0],
        y=Ag_15ml[:, 2],
        # name="Ag(111) 15 ML",
        showlegend=False,
        mode="markers",
        marker=dict(symbol=exciton, size=14, color="grey"),
    )
)

# xval = 10
# yval = (5, 150)
# fig.add_shape(
#     go.layout.Shape(
#         type="line",
#         x0=xval,
#         x1=xval,
#         y0=yval[0],
#         y1=yval[1],
#         line=dict(color="gold", width=14),
#         opacity=0.25,
#     )
# )

# xval = 10
# yval = (20, 150)
# fig.add_shape(
#     go.layout.Shape(
#         type="line",
#         x0=xval,
#         x1=xval,
#         y0=yval[0],
#         y1=yval[1],
#         line=dict(color="gold", width=14),
#         opacity=0.25,
#     )
# )

fig.add_trace(
    go.Scatter(
        x=Au_10ml[:, 0],
        y=Au_10ml[:, 2],
        name="Au(111)",
        mode="markers",
        marker=dict(symbol=exciton, size=14, color="gold"),
    )
)


fig.update_layout(
    yaxis_range=(np.log10(0.01), np.log10(190)), xaxis_range=(0, 20)
)  # yaxis_range=(0, 3),  margin=dict(t=0)
fig.update_xaxes(minor_ticks="inside")
fig.update_yaxes(type="log", minor_ticks="inside")
fig.update_layout(
    legend=dict(
        x=0.22,  # x position (1 is the far right)
        y=0.16,  # y position (0 is the far bottom)
        xanchor="right",  # Horizontal alignment: 'left', 'right', 'center', 'auto'
        yanchor="top",  # Vertical alignment: 'top', 'bottom', 'middle', 'auto'
        # bordercolor="Black",  # Border color of the legend
        # borderwidth=2,  # Border width of the legend
        # bgcolor="rgba(230, 230, 150, 0.8)",  # Background color of the legend with transparency
        bgcolor=None,
        font=dict(size=18),
    ),
)
fig.show()

fig.write_image(
    r"C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Figs\CH6_tr_arpes\exciton_decay_comparison.png"
)